# 3 - Faster Sentiment Analysis

이전 노트에서 우리는 감성분석의 다른 테크닉들을 사용해서 ~84%까지의 정확도를 얻는 데 성공했다. 이번 노트에서는 "FastText" model을 사용해서 파라미터 수는 절반 정도이면서 상당히 빠른 모델을 실행해 볼 것이다.

## Preparing Data

FastText paper의 핵심 개념 중 한가지는 n-gram의 입력 문장과 이것들을 한 문장의 마지막에 추가하는 것인데 여기서 우리는 bi-gram을 사용할 것이다. 간략하게 말하자면 bi-gram은 한 문장에서 연속적으로 나타나는 단어들/토큰들의 짝이다.

가령 "How are you ?"이면 바이그램은 "how are" "are you" "you ?" 가 된다.

generate_bigrams 함수는 이미 토큰화된 문장을 가지고 바이그램을 계산하고 토큰화된 리스트의 마지막에 추가한다

In [24]:
def generate_bigrams(x):
    n_grams = set(zip(*[x[i:] for i in range(2)]))
    for n_gram in n_grams:
        x.append(' '.join(n_gram))
    return x

예를 들어

In [25]:
generate_bigrams(['This' , 'film' , 'is' , 'terrible'])

['This', 'film', 'is', 'terrible', 'This film', 'film is', 'is terrible']

RNN을 사용하는 것이 아니기 때문에 packed padded sentence를 사용할 필요가 없으므로
include_length = True 를 할 필요는 없다

In [30]:
import torch
from torchtext import data
from torchtext import datasets

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', preprocessing = generate_bigrams)
LABEL = data.LabelField(dtype = torch.float)

이전에 했던것 처럼 IMDB 데이터를 받아서 셋을 분리한다

In [33]:
import random

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

vocab를 만들고 사전 학습된 워드 임베딩을 불러온다

In [34]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [35]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

## Build the Model

이 모델은 레이어가 임베딩 레이어 그리고 선형 레이어 2개밖에 되지 않기 때문에 파라미터가 훨씬 적다.

우리는 avg_pool2d 함수를 가지고 averaging을 한다. 왜 문장들은 1차원이지만 2차원의 풀링을 하는지에 대해 의문스러울 수 있지만 한 축이 단어들이라면 나머지 한축은 단어의 임베딩이 들어가기 때문이다.

아래 그림에서는 5차원의 워드 임베딩을 하고 필터가 첫번쨰 열부터 오른쪽으로 이동하면서 임베딩값들의 평균을 계속해서 연산하고 결과적으로는 1X5의 텐서를 반환하며 이 텐서는 이제 선형 레이어로 들어가 예측값을 만들게 된다.

In [36]:
import torch.nn as nn
import torch.nn.functional as F

class FastText(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        
        self.fc = nn.Linear(embedding_dim, output_dim)
        
    def forward(self, text):
        
        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
                
        #embedded = [sent len, batch size, emb dim]
        
        embedded = embedded.permute(1, 0, 2)
        
        #embedded = [batch size, sent len, emb dim]
        
        pooled = F.avg_pool2d(embedded, (embedded.shape[1], 1)).squeeze(1) 
        
        #pooled = [batch size, embedding_dim]
                
        return self.fc(pooled)

이전처럼, 우리는 FastText의 클래스의 인스턴스를 생성할 것이다

In [37]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
OUTPUT_DIM = 1
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = FastText(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM, PAD_IDX)

In [38]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,500,301 trainable parameters


In [39]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-1.5268,  1.5639, -1.0541,  ...,  1.0045, -0.6813, -0.8846],
        [ 1.2275,  2.9389,  0.9619,  ...,  0.8306,  1.3495, -0.2619],
        [-0.1328, -0.1483,  0.7629,  ...,  1.4513,  0.6356,  1.0246]])

값을 모르고 패딩된 토큰에 대해서는 초기 가중치를 제로로 해준다

In [42]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

# Train the model

모델을 트레이닝 하는 과정은 기존과 같다
일단 옵티마이징을 시작하자..

In [43]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [44]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [45]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [48]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [49]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [50]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [51]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 12s
	Train Loss: 0.687 | Train Acc: 58.82%
	 Val. Loss: 0.639 |  Val. Acc: 68.22%
Epoch: 02 | Epoch Time: 0m 11s
	Train Loss: 0.649 | Train Acc: 73.16%
	 Val. Loss: 0.525 |  Val. Acc: 74.62%
Epoch: 03 | Epoch Time: 0m 9s
	Train Loss: 0.579 | Train Acc: 78.96%
	 Val. Loss: 0.451 |  Val. Acc: 79.21%
Epoch: 04 | Epoch Time: 0m 9s
	Train Loss: 0.505 | Train Acc: 83.50%
	 Val. Loss: 0.401 |  Val. Acc: 82.96%
Epoch: 05 | Epoch Time: 0m 8s
	Train Loss: 0.439 | Train Acc: 86.39%
	 Val. Loss: 0.378 |  Val. Acc: 85.33%


In [52]:
model.load_state_dict(torch.load('tut3-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.389 | Test Acc: 84.92%


In [53]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = generate_bigrams([tok.text for tok in nlp.tokenizer(sentence)])
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [54]:
predict_sentiment(model, "This film is terrible")

8.928639078087386e-12

In [57]:
predict_sentiment(model, "This film is great")

1.0

다음 노트에서는 CNN을 이용하여 감성 분석을 할 것이다.